# 🐳 Docker Compose: App Python con MySQL

Guida completa passo-passo con spiegazione dettagliata del file `docker-compose.yml`.

## 🎯 Obiettivo

Costruire un'app Python che:

- si connette a un database MySQL in un container separato,

- è gestita da Docker Compose.


## 📁 Struttura del progetto

```
docker-compose-python-mysql/
│
├── app2.py               # Codice Python
├── requirements.txt      # Dipendenze pip
├── Dockerfile            # Dockerfile per Python app
└── docker-compose.yml    # File principale Docker Compose
```

# Configurazione nel contesto dell'Ambiente virtuale

## Creazione di un ambiente virtuale
Per isolare le dipendenze del progetto, puoi creare un ambiente virtuale utilizzando il seguente comando:
```bash
python -m venv env
```

## Attivazione dell'ambiente virtuale
Una volta creato l'ambiente virtuale, attivalo con il comando appropriato per il tuo sistema operativo:
- **Linux/Mac**: 
    ```bash
    source env/bin/activate
    ```
- **Windows**: 
    ```bash
    env\Scripts\activate
    ```

## Installazione delle librerie
Dopo aver attivato l'ambiente virtuale, puoi installare le librerie che servono al suo interno utilizzando a esempio:
```bash
pip install mysql-connector-python
```

### Controllo della versione di Python
```bash
python --version
```

### Controllo delle librerie installate
Per vedere tutte le librerie installate e le loro versioni, usa:
```bash
pip list
```

In [ ]:
#Se vuoi partire da un repository git
!git clone [url]

## 1. ✍️ app2.py – Codice Python

In [1]:
%%writefile app2.py

import mysql.connector
import time

def connect():
    for _ in range(10):
        try:
            conn = mysql.connector.connect(
                host="mysql",
                user="user",
                password="password",
                database="testdb"
            )
            print("✅ Connessione a MySQL riuscita!")
            cursor = conn.cursor()
            cursor.execute("CREATE TABLE IF NOT EXISTS saluti (id INT AUTO_INCREMENT PRIMARY KEY, messaggio VARCHAR(255))")
            cursor.execute("INSERT INTO saluti (messaggio) VALUES ('Ciao da Docker Compose con MySQL')")
            conn.commit()
            cursor.execute("SELECT * FROM saluti")
            for row in cursor.fetchall():
                print(row)
            conn.close()
            break
        except mysql.connector.Error as err:
            print("⏳ Tentativo di connessione fallito. Ritento...")
            time.sleep(2)

if __name__ == "__main__":
    connect()


Writing app2.py


## 2. 📦 Creare il file `requirements.txt`

In [ ]:
!pip freeze > requirements.txt

## 3. 🛠️ Creare il Dockerfile

In [4]:
dockerfile = '''
FROM python:latest

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY app2.py .

CMD ["python", "app2.py"]
'''

with open("Dockerfile", "w") as f:
    f.write(dockerfile)

print("✅ Dockerfile creato.")


✅ Dockerfile creato.


## 📌 Cos’è `docker-compose.yml`?

È un file YAML (indentazione molto importante!) che:
- definisce **servizi** (container),
- configura **volumi**, **reti**, **variabili d’ambiente**, **dipendenze**,
- permette di avviare tutto con un solo comando:  
  ```bash
  docker-compose up --build
  ```

## 🧱 File `docker-compose.yml` spiegato riga per riga


In [5]:
dockerfile = '''
services:
    app:
      build: .
      depends_on:
        - mysql
      networks:
        - rete_app
    mysql:
      image: mysql:latest
      restart: always
      environment:
        MYSQL_ROOT_PASSWORD: rootpass
        MYSQL_DATABASE: testdb
        MYSQL_USER: user
        MYSQL_PASSWORD: password
      ports:
        - "3306:3306"
      networks:
        - rete_app
networks:
  rete_app:
    driver: bridge
'''

with open("docker-compose.yml", "w") as f:
    f.write(dockerfile)

print("✅ Dockerfile creato.")

✅ Dockerfile creato.


### 🔧 Definizione dei servizi

```yaml
services:
```

Ogni container è un **servizio**. Definiamo qui sia `app` (Python) sia `mysql`.

### 🐍 Servizio `app` (contenitore Python)

```yaml
  app:
    build: .
    depends_on:
      - mysql
    networks:
      - rete_app
```

- `build: .`: costruisce l'immagine a partire dal `Dockerfile`.
- `depends_on`: MySQL viene avviato prima.
- `networks`: l'app è connessa alla rete `rete_app` per comunicare con MySQL.


### 🐬 Servizio `mysql`

```yaml
  mysql:
    image: mysql:latest
    restart: always
    environment:
      MYSQL_ROOT_PASSWORD: rootpass
      MYSQL_DATABASE: testdb
      MYSQL_USER: user
      MYSQL_PASSWORD: password
    ports:
      - "3306:3306"
    networks:
      - rete_app
```

- `image`: usa l'immagine ufficiale MySQL 8.
- `restart`: si riavvia automaticamente in caso di errore.
- `environment`: imposta root, utente, password e database iniziali.
- `ports`: mappa la porta locale 3306 alla porta interna.
- `networks`: lo collega alla stessa rete del servizio `app`.


### 🌐 Rete

```yaml
networks:
  rete_app:
    driver: bridge
```

- Crea una rete isolata tra `app` e `mysql`.


## 🖥️ Comandi Docker Compose

### 1. 🔨 Costruire e avviare tutto

```bash
docker-compose up --build
```

- Costruisce l’immagine Python (dal `Dockerfile`)
- Avvia MySQL e l'app
- Mostra i log live


### 2. 🛑 Fermare e rimuovere tutto

```bash
docker-compose down
```

- Ferma i container
- Rimuove i volumi temporanei
- Libera le porte

### 3. 📂 Eseguire in background (opzionale)

```bash
docker-compose up -d
```

- Avvia i container in **detached mode** (background).

[documentazione](https://docs.docker.com/compose/gettingstarted/)